In [1]:
import matplotlib.pyplot as plt
import numpy as np

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import HTML

In [2]:
# universal constants
h=6.626*10**(-34) #plank's constant J.s
me=9.109*10**(-31) #electron's mass kg
mecsquared=511 # electron's rest mass in keV
electron=1.602*10**(-19)# electron's charge

In [3]:
#electron's wavelength in m for an energy in keV
def lamb(Energy):
    l=h/np.sqrt(2*me*Energy*1000*electron)*(1+Energy/(2*mecsquared))**(-0.5)
    return l

In [4]:
#check (should be 2.5 pm)
lamb(200)

2.5080989936058438e-12

In [5]:
# phase contrast transfer function 
# u reciprocal length
# deltaf: defocus
# cs:spherical aberration
# lamb: wavelength
def pctf(u,deltaf,cs,lamb):
    chi=np.pi*deltaf*lamb*u**2+0.5*np.pi*cs*lamb**3*u**4
    tf=np.sin(chi)
    return tf

In [6]:
# phase contrast transfer function squared (to compare w diffractogram)

def pctfsq(u,deltaf,cs,lamb):
    chi=np.pi*deltaf*lamb*u**2+0.5*np.pi*cs*lamb**3*u**4
    tf=(np.sin(chi))**2
    return tf

In [7]:
# 2-d PCTF. Only interesting whn astigmatism is present
def pctf2d(ux,uy,deltaf,astig,cs,lamb):
    chi=np.pi*(deltaf+astig/2)*lamb*(ux**2)+np.pi*(deltaf-astig/2)*lamb*(uy**2)+0.5*np.pi*cs*lamb**3*(ux**2+uy**2)**2
    tf=np.sin(chi)
    return tf

## Temporal envelope
$$E(u)=\exp({-\frac{(\pi\Delta\lambda u^2)^2}{16 \ln(2)}})$$
with
$$\Delta = C_c\frac{\Delta E}{E_0}$$
$\Delta E$ energy spread

$E_0$, incident energy

In [8]:
# temporal envelope, as defined in Spence book
def temporal_envelope(Energy,u,cc,Espread):
    lam=lamb(Energy)
    delta=cc*Espread/(1000*Energy)
    te=np.exp(-(np.pi*delta*lam*u**2)**2/(16*np.log(2)))
    #te=np.exp(-(np.pi*delta*lam*u**2)**2/(4))
    return te

In [9]:
def scherzer(cs,Energy):
    lam=lamb(Energy)
    sch=-(4*cs*10**(-6)*lam/3)**(0.5)
    return sch

In [10]:
# Check. Should be -63 nm
scherzer(1200,200)

-6.334791543349591e-08

In [11]:
# Interactive plot of the PCTF
def pctfplot(deltaf,cs, Energy,cc,Espread):
    lam=lamb(Energy)
    u = np.linspace(0,10,2000)
    te=temporal_envelope(Energy,10**9*u,cc*10**(-3),Espread)
    plt.plot(u,te*pctf(10**9*u,deltaf*10**(-9),cs*10**(-6),lam))
#    plt.axhline(0)
    plt.grid(True)
    plt.plot(u,te)
    plt.show()

interact(pctfplot, deltaf=widgets.IntSlider(min=-200,max=100,step=1,value=-67,description='Defocus in nm'),
         cs=widgets.IntSlider(min=-100,max=1500,step=5,value=1200,description='Cs in microns'),
         Energy=widgets.IntSlider(min=20,max=300,step=10,value=200,description='Energy in keV'),
         cc=widgets.FloatSlider(min=0,max=3,step=0.1,value=1.1,description='Cc in mm'),
         Espread=widgets.FloatSlider(min=0,max=8,step=0.1,value=2.0,description='Energy spread in eV'))  ; 


interactive(children=(IntSlider(value=-67, description='Defocus in nm', min=-200), IntSlider(value=1200, descr…

In [12]:
# Interactive plot of the PCTF squared
def pctfsqplot(deltaf,cs, Energy,cc,Espread):
    lam=lamb(Energy)
    u = np.linspace(0,10,2000)
    te=temporal_envelope(Energy,10**9*u,cc*10**(-3),Espread)
    plt.plot(u,te*pctfsq(10**9*u,deltaf*10**(-9),cs*10**(-6),lam))
#    plt.axhline(0)
    plt.grid(True)
    plt.plot(u,te)
    plt.show()

interact(pctfsqplot, deltaf=widgets.IntSlider(min=-200,max=150,step=1,value=-67,description='Defocus in nm'),
         cs=widgets.IntSlider(min=-100,max=1500,step=5,value=1200,description='Cs in microns'),
         Energy=widgets.IntSlider(min=20,max=300,step=10,value=200,description='Energy in keV'),
         cc=widgets.FloatSlider(min=0,max=3,step=0.1,value=1.1,description='Cc in mm'),
         Espread=widgets.FloatSlider(min=0,max=8,step=0.1,value=2.0,description='Energy spread in eV'))  ; 

interactive(children=(IntSlider(value=-67, description='Defocus in nm', max=150, min=-200), IntSlider(value=12…

In [13]:
def pctfplot2d(deltaf,cs,astig, Energy,cc,Espread):
    ux = np.linspace(-10, 10, 301)
    uy = np.linspace(-10, 10, 301)
    X, Y = np.meshgrid(ux, uy)
    Z = np.sqrt(X**2 + Y**2)
    lam=lamb(Energy)
    te=temporal_envelope(Energy,10**9*Z,cc*10**(-3),Espread)
    ZZ=te*pctf2d(X*10**9,Y*10**9,deltaf*10**(-9),astig*10**(-9),cs*10**(-6),lam)
    plt.figure()
    cp = plt.contourf(X, Y, ZZ)
    plt.colorbar(cp)
    plt.axes().set_aspect('equal')
    plt.xlabel('ux (1/nm)')
    plt.ylabel('uy (1/nm)')
    plt.set_cmap('coolwarm') 
# plt.set_cmap('gray')
    plt.show()
    
#pctfplot2d(-60,1000,200) 
interact_manual(pctfplot2d, deltaf=widgets.IntSlider(min=-100,max=100,step=1,value=-67,description='Defocus in nm'),
         cs=widgets.IntSlider(min=-100,max=1500,step=5,value=1200,description='Cs in microns'),
         Energy=widgets.IntSlider(min=20,max=300,step=10,value=200,description='Energy in keV'),
         astig=widgets.IntSlider(min=-300,max=300,step=10,value=0,description='astigmatism in nm'),       
         cc=widgets.FloatSlider(min=0,max=3,step=0.1,value=1.1,description='Cc in mm'),
         Espread=widgets.FloatSlider(min=0,max=8,step=0.1,value=2.0,description='Energy spread in eV')
         );
#defocus in nm; Cs in microns and beam energy in keV

interactive(children=(IntSlider(value=-67, description='Defocus in nm', min=-100), IntSlider(value=1200, descr…